# A minimal form

We veranderen de code  aan de `detail.html` door met een form element te werken.


In [ ]:
<form action="{% url 'polls:vote' question.id %}" method="post">
    {% csrf_token %}
    <fieldset>
        <legend>
            <h1>{{question.question_text}}</h1>
        </legend>
        {% if error_message %} 
            <p><strong>{{error_message}}</strong></p>
        {% endif %}

        {% for choice in question.choice_set.all %}
            <input type="radio" name="choice" id="choice{{ forloop.counter }}" value="{{ choice.id }}">
            <label for="choice{{ forloop.counter }}">{{choice.choice_text}}</label>
            <br>
        {% endfor %}
            
    </fieldset>
    <input type="submit" value="Vote">
</form>

Je kan via developer tools zien hoe de bovenstaande code wordt omgezet naar HTML.

je krijgt voor elke choice een raxdio-button.

<form> heeft als method POST. (je kan ook nog de method GET hebben)
POST => stuurt data naar de server

de tag ``{% csrf_token %}``  is verplicht toe te voegen. Je kan via "developer tools => application => cookies" de crsftoken zien. Deze dient om users te beschermen tegen "Cross Site Request Forgeries" wat bij "POST" zou kunnen gebeuren.

de "name" van elke keuze is "choice". Als iemand de radiobutton kiest en op de knop drukt, dan zal deze de "POST" data  "choice=#" doorsturen. Met # == aan de value van de keuze.


de form heeft als action = "{% url 'polls:vote' question.id %}"
dit zal na het indienen (submit) van het formulier naar dit adres gaan.

{{ forloop.counter }} => houdt een counter bij van de iteraties van de loop.

De url voor onze "vote" bestaat al.

Nu maken we nog de view voor onze vote.

Vooraleer we verder gaan, zullen we eerst onze request eens onderzoeken. 
We beginnen met de request zelf.
Voor meer info check `https://docs.djangoproject.com/en/4.2/ref/request-response/`

In [ ]:
def vote(request, question_id):
    try:
        print(request)
    
        print(request.scheme)

        print("body:", request.body)

        print(request.GET)
        print(request.POST) # is een object zoals een dictionary
        print(request.POST["choice"])

        print("method:", request.method)

        print("encoding:", request.encoding)

        print("headers - useragent:", request.headers["User-Agent"])
    except:
        print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")

    return HttpResponse(f"You're voting on question {question_id}.")

Nu updaten we onze functie zodat we de vote kunnen aanpassen in onze database.

Eerst importeren we nog een aantal extra functies, klasses:

In [ ]:
from django.urls import reverse
from django.http import HttpResponseRedirect


Nu passen we de view aan:

We beginnen met de redirect toe te voegen aan de vote.
Deze gebruiken we om op dezelde manier als binnen de templates een dynamische verwijzing naar bepaalde urls te maken.
We moeten de urls niet hard-coderen.

In [ ]:
def vote(request, question_id):

    return HttpResponseRedirect(reverse('polls:results', args=(question_id,)))

Eerst zoeken we de juiste data uit de DB. (Hoe?)

Hoe kunnen we alle choices van onze question te weten komen ?

In [ ]:
def vote(request, question_id):
    question = get_object_or_404(Question, pk = question_id)
    print(question.choice_set.all())

    return HttpResponseRedirect(reverse('polls:results', args=(question_id,)))


Nu proberen we enkel die choice te kiezen die we in onze form hebben gekozen.


In [ ]:
def vote(request, question_id):
    question = get_object_or_404(Question, pk = question_id)
    selected_choice = question.choice_set.get(pk=request.POST["choice"])
    print(selected_choice)
    
    return HttpResponseRedirect(reverse('polls:results', args=(question_id,)))

Dit werkt niet wanneer we geen keuze maken en enkel op de vote knop drukken. 

Hoe kunnen we dit oplossen?

In [ ]:
def vote(request, question_id):
    question = get_object_or_404(Question, pk = question_id)
    try:
        selected_choice = question.choice_set.get(pk=request.POST["choice"])
    except (KeyError, Choice.DoesNotExist):
        return HttpResponse(f"You're voting on question {question_id}.")
    return HttpResponseRedirect(reverse('polls:results', args=(question_id,)))

request.POST['choice'] zal een `KeyError` oproepen wanneer 'choice' niet voorzien werd in de POST data.

door een try toe te voegen zal de "selected_choice" enkel worden gekozen wanneer er ook een "request.POST['choice']" is.

Wanneer we geen keuze maken sturen we een text naar onze pagina.

Dit laatste willen we veranderen naar de logica dat:
* als we geen keuze maken => blijven we op de pagina

In [ ]:
def vote(request, question_id):
    question = get_object_or_404(Question, pk = question_id)
    try:
        selected_choice = question.choice_set.get(pk=request.POST["choice"])
    except (KeyError, Choice.DoesNotExist):
        context = {
            "question" : question
        }
        return render(request, "polls/detail.html", context)
    

    return HttpResponseRedirect(reverse('polls:results', args=(question_id,)))

Een andere manier is via `redirect()`

In [ ]:
from django.shortcuts import redirect



def vote(request, question_id):
    question = get_object_or_404(Question, pk = question_id)
    try:
        selected_choice = question.choice_set.get(pk=request.POST["choice"])
    except (KeyError, Choice.DoesNotExist):
        return redirect("polls:detail", question_id)
    
    return HttpResponseRedirect(reverse('polls:results', args=(question_id,)))

Het nadeel t.o.v. de vorige methode is dat we nu zowel in deze view (vote) als in de "detail"-view opnieuw de Question gaan halen in de DB.

Daarom is de vorige dus eigenlijk beter.

Nu willen we onze choice saven in de DB.  Hoe doen we dat ?

In [ ]:
def vote(request, question_id):
    question = get_object_or_404(Question, pk = question_id)
    try:
        selected_choice = question.choice_set.get(pk=request.POST["choice"])
    except (KeyError, Choice.DoesNotExist):
        context = {
            "question" : question
        }
        return render(request, "polls/detail.html", context)
    
    selected_choice.votes += 1
    selected_choice.save()
    return HttpResponseRedirect(reverse('polls:results', args=(question_id,)))

Nu dat de `vote()` volledig is, maken we de `results()` view en de `results.html` -file.

In [ ]:
def results(request, question_id):
    question = get_object_or_404(Question, pk=question_id)
    context = {"question" : question}
    return render(request, "polls/results.html", context)

Deze view verschilt amper van de "detail" view.

Deze redundantie zullen we later proberen oplossen.

In [ ]:
<h1>{{ question.question_text }}</h1>

<ul>
    {% for choice in question.choice_set.all %}
        <li>
            {{ choice.choice_text }} -- {{ choice.votes }} vote{{ choice.votes|pluralize }}
        </li>
    {% endfor %}    
</ul>

<a href="{% url 'polls:detail' question.id %}"> Vote again ? </a>

De `|pluralize` in `{{ choice.votes|pluralize }}` is een built-in template tag. 

Je kan de werking hiervan terugvinden op `https://docs.djangoproject.com/en/4.2/ref/templates/builtins/#pluralize`.

Je kan ook verschillende andere built-in template tags terugvinden. Deze tags kunnen extra functionaliteit voorzien in de template language.

## Generic Views

zowel `detail()`, `results()` als `index()` zijn zeer kort.

Het zijn in feite een type van views dat regelmatig wordt gebruikt. 
Daarvoor bestaan dan ook generieke views die in Django zijn voorzien. (klasses)

We passen onze views aan:

In [ ]:
from django.views import generic


class IndexView(generic.ListView):
    template_name = "polls/index.html"
    context_object_name = "latest_question_list"

    def get_queryset(self) -> QuerySet[Any]:
        return Question.objects.order_by("-pub_date")[:3]

class DetailView(generic.DetailView):
    model = Question
    template_name = "polls/detail.html"

class RestultsView(generic.DetailView):
    model = Question
    template_name = "polls/results.html"

We moeten ook nog onze urls aanpassen:

In [ ]:
from django.urls import path
from . import views

app_name = "polls"
urlpatterns = [
    path("", views.IndexView.as_view(), name="index"),
    path("<int:question_id>/", views.DetailView.as_view(), name="detail"),
    path("<int:question_id>/results/", views.RestultsView.as_view(), name="results"),
    path("<int:question_id>/vote/", views.vote, name="vote"),
    path("test_db", views.test_db, name="test_db"),
]

we gebruiken de klasse-views, waarachter we wel `as_view()` moeten gebruiken. 

We proberen nu ons programma te runnen. Waaromlukt dit niet ?

In [ ]:
raise AttributeError(
AttributeError: Generic detail view RestultsView must be called with either an object pk or a slug in the URLconf.

We moeten onze question_id vervangen door de pk

In [ ]:
path("<int:pk>/", views.DetailView.as_view(), name="detail"),
path("<int:pk>/results/", views.RestultsView.as_view(), name="results"),

Bij een `DetailView` is de default template: "<app_name>/<model_name>_detail.html", dus bij onze `detail()` & `result()` -view zou dat dus `polls/question_detail.html` zijn. We overschrijven `template_name` hiervoor.

Er wordt automatisch "question" toegevoegd aan de context. Dat is de naam van het model.

Bij de ListView is dat "<model_name>_list" dus zou hier "question_list" zijn. Aangezien we in onze template al latest_question_list hebben gebruikt, overschrijven we deze door `context_object_name` te overschrijven.


Zowel `template_name` als `context_object_name` kunnen in alle Generic Views worden overschreven.
